In [127]:
%matplotlib inline
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import prettytensor as pt

In [128]:
rows, cols = 70, 100
img_shape = (rows, cols)
img_flat_size = rows * cols
img_classes = 3
img_channels = 3

In [129]:
resized_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/ISIC-2017/resized/"

In [130]:
isic_pickle_file = resized_folder_path + "ISIC.pickle"
with open(isic_pickle_file, "rb") as f:
    save = pickle.load(f)
    train_dataset =  save["train_dataset"]
    train_labels =  save["train_labels"]
    test_dataset =  save["test_dataset"]
    test_labels =  save["test_labels"]
    del save
    print("Training Set {0} {1}".format(train_dataset.shape, train_labels.shape))
    print("Test Set {0} {1}".format(test_dataset.shape, test_labels.shape))
test_labels[0:10, :]

Training Set (1688, 70, 100, 3) (1688, 3)
Test Set (300, 70, 100, 3) (300, 3)


array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]])

In [112]:
# input_x = tf.placeholder(tf.float32, [None, img_flat_size], name = "input_x") 
input_layer = tf.placeholder(tf.float32, [None, rows, cols, img_channels], name = "input_layer")
y_true = tf.placeholder(tf.float32, [None, img_classes], name = "y_true")
y_true_cls = tf.argmax(y_true, dimension = 1)

In [113]:
x_pretty = pt.wrap(input_layer)
with pt.defaults_scope(activation_fn = tf.nn.relu):
    y_pred, loss = x_pretty.\
        conv2d(kernel = 5, depth = 16, name = "conv_layer_1").\
        max_pool(kernel = 2, stride = 2).\
        conv2d(kernel = 5, depth = 36, name = "conv_layer_2").\
        max_pool(kernel = 2, stride = 2).\
        flatten().\
        fully_connected(size = 128, name = "fc_layer_1").\
        softmax_classifier(num_classes = img_classes, labels = y_true)

In [114]:
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-4).minimize(loss)

In [115]:
y_pred_cls = tf.argmax(y_pred, dimension = 1)

In [116]:
correct_pred = tf.equal(y_true_cls, y_pred_cls)

In [117]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [118]:
sess = tf.Session()

In [119]:
sess.run(tf.global_variables_initializer())

In [120]:
train_batch_size = 64

In [121]:
total_iteration = 0

def optimize(num_iteration):
    global total_iteration
    for i in range(total_iteration, total_iteration + num_iteration):
        range_from = min(i - total_iteration, len(train_dataset))
        range_to = min((range_from + train_batch_size), len(train_dataset))
        if range_from == len(train_dataset):
            break
#         print("{0} - {1}".format(range_from, range_to))
        x_train = train_dataset[range_from:range_to, :, :, :]
        y_train = train_labels[range_from:range_to,:]
        feed_dict_train = {
            input_layer: x_train,
            y_true: y_train
        }
        
        sess.run(optimizer, feed_dict = feed_dict_train)
        

    feed_dict_test = {
        input_layer: test_dataset,
        y_true: test_labels
    }
    acc = sess.run(accuracy, feed_dict = feed_dict_test)
    print("Accuracy {:.2%}".format(acc))

    total_iteration += num_iteration
    print("Total Iteration {0}".format(total_iteration))

In [122]:
optimize(10)

Accuracy 73.33%
Total Iteration 10


In [123]:
optimize(100)

Accuracy 61.67%
Total Iteration 110


In [124]:
optimize(10)

Accuracy 63.67%
Total Iteration 120


In [125]:
optimize(10)

Accuracy 66.67%
Total Iteration 130


In [126]:
optimize(30)

Accuracy 61.33%
Total Iteration 160
